# 根据论文标题获取固定的引用格式

可以避免手动查询，自动化完成任务，但不保证正确~

默认查询返回列表中排第一个位置的论文，仅进行了少量测试，此脚本仅供分享交流。

食用方法：把需要转换的标题按行排列，复制到titles中，执行此脚本即可。

In [1]:
from lxml import html
from urllib.parse import quote_plus, quote, urlencode, parse_qs
import requests
import json
from IPython.display import HTML
import time
import re

# formation in: sc_MLA, sc_APA, sc_GBT7714
def query_baidu_xueshu(title, formation='sr_APA'):

    headers= {
        "User-Agent":"Mozilla/5.0 (iPhone; CPU iPhone OS 16_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.6 Mobile/15E148 Safari/604.1 Edg/119.0.0.0",
        "Referer": "https://xueshu.baidu.com/"
    }
    
    url = "https://xueshu.baidu.com/s?"
    params = {"wd": title,}
    _url = url + urlencode(params)
    print(_url)
    
    page = requests.get(_url, headers=headers)
    #print(page.content)
    tree = html.fromstring(page.content)
    
    m = re.search(r"window.location.replace\(\\'//xueshu.baidu.com/usercenter/paper/show\?paperid=([a-zA-Z_0-9]*)&", str(page.content))
    
    if m is not None:
        pid = m.group(0).split('paperid=')[1][:-1]
    else:
        papers = tree.xpath('//div[@class="sc_content"]/h3/a')
        href = papers[0].get('href').split('?', 2)[1]
        pid = parse_qs(href)['paperid'][0]
    
    _url = f"https://xueshu.baidu.com/u/citation?type=cite&paperid={pid}"
    print(_url)
    
    cite = requests.get(_url, headers=headers)
    #print(cite.text)
    
    apa = json.loads(cite.text)['data']['formation'][4:]

    return apa.replace('#i{', '<i>').replace('}', '</i>')


In [2]:
title = '143ND/144ND AND 87SR/86SR RATIOS IN RECENT MAGMATIC ROCKS OF THE KURILE ISLAND ARC'

apa = query_baidu_xueshu(title)

HTML(apa)

https://xueshu.baidu.com/s?wd=143ND%2F144ND+AND+87SR%2F86SR+RATIOS+IN+RECENT+MAGMATIC+ROCKS+OF+THE+KURILE+ISLAND+ARC
https://xueshu.baidu.com/u/citation?type=cite&paperid=4f38a87a6cbb0fe8469a8e6a76522da2


In [3]:
titles = """
143ND/144ND AND 87SR/86SR RATIOS IN RECENT MAGMATIC ROCKS OF THE KURILE ISLAND ARC
143ND/144ND AND 87SR/86SR RATIOS IN RECENT MAGMATIC ROCKS OF THE KURILE ISLAND ARC
A RHYOLITE TO DACITE SEQUENCE OF VOLCANISM DIRECTLY FROM THE HEATED LOWER CRUST: LATE PLEISTOCENE TO HOLOCENE NUMAZAWA VOLCANO, NE JAPAN
A RHYOLITE TO DACITE SEQUENCE OF VOLCANISM DIRECTLY FROM THE HEATED LOWER CRUST: LATE PLEISTOCENE TO HOLOCENE NUMAZAWA VOLCANO, NE JAPAN
A RHYOLITE TO DACITE SEQUENCE OF VOLCANISM DIRECTLY FROM THE HEATED LOWER CRUST: LATE PLEISTOCENE TO HOLOCENE NUMAZAWA VOLCANO, NE JAPAN
A RHYOLITE TO DACITE SEQUENCE OF VOLCANISM DIRECTLY FROM THE HEATED LOWER CRUST: LATE PLEISTOCENE TO HOLOCENE NUMAZAWA VOLCANO, NE JAPAN
A RHYOLITE TO DACITE SEQUENCE OF VOLCANISM DIRECTLY FROM THE HEATED LOWER CRUST: LATE PLEISTOCENE TO HOLOCENE NUMAZAWA VOLCANO, NE JAPAN
A RHYOLITE TO DACITE SEQUENCE OF VOLCANISM DIRECTLY FROM THE HEATED LOWER CRUST: LATE PLEISTOCENE TO HOLOCENE NUMAZAWA VOLCANO, NE JAPAN
A RHYOLITE TO DACITE SEQUENCE OF VOLCANISM DIRECTLY FROM THE HEATED LOWER CRUST: LATE PLEISTOCENE TO HOLOCENE NUMAZAWA VOLCANO, NE JAPAN
A TRIASSIC TO JURASSIC ARC IN NORTH BORNEO: GEOCHRONOLOGY, GEOCHEMISTRY, AND GENESIS OF THE SEGAMA VALLEY FELSIC INTRUSIONS AND THE SABAH OPHIOLITE
A TRIASSIC TO JURASSIC ARC IN NORTH BORNEO: GEOCHRONOLOGY, GEOCHEMISTRY, AND GENESIS OF THE SEGAMA VALLEY FELSIC INTRUSIONS AND THE SABAH OPHIOLITE
A TRIASSIC TO JURASSIC ARC IN NORTH BORNEO: GEOCHRONOLOGY, GEOCHEMISTRY, AND GENESIS OF THE SEGAMA VALLEY FELSIC INTRUSIONS AND THE SABAH OPHIOLITE
A TRIASSIC TO JURASSIC ARC IN NORTH BORNEO: GEOCHRONOLOGY, GEOCHEMISTRY, AND GENESIS OF THE SEGAMA VALLEY FELSIC INTRUSIONS AND THE SABAH OPHIOLITE
A U-SERIES STUDY OF LAVAS FROM KAMCHATKA AND THE ALEUTIANS: CONSTRAINTS ON SOURCE COMPOSITION AND MELTING PROCESSES
A U-SERIES STUDY OF LAVAS FROM KAMCHATKA AND THE ALEUTIANS: CONSTRAINTS ON SOURCE COMPOSITION AND MELTING PROCESSES
A U-SERIES STUDY OF LAVAS FROM KAMCHATKA AND THE ALEUTIANS: CONSTRAINTS ON SOURCE COMPOSITION AND MELTING PROCESSES
ACROSS‐ARC DIVERSITY IN RHYOLITES FROM AN INTRA‐OCEANIC ARC: EVIDENCE FROM IODP SITE U1437, IZU‐BONIN REAR‐ARC AND SURROUNDING AREA
ACROSS‐ARC DIVERSITY IN RHYOLITES FROM AN INTRA‐OCEANIC ARC: EVIDENCE FROM IODP SITE U1437, IZU‐BONIN REAR‐ARC AND SURROUNDING AREA
ACROSS‐ARC DIVERSITY IN RHYOLITES FROM AN INTRA‐OCEANIC ARC: EVIDENCE FROM IODP SITE U1437, IZU‐BONIN REAR‐ARC AND SURROUNDING AREA
ACROSS-ARC GEOCHEMICAL TRENDS IN THE IZU-BONIN ARC: CONTRIBUTIONS FROM THE SUBDUCTING SLAB
ACROSS-ARC GEOCHEMICAL VARIATION OF QUATERNARY LAVAS IN WEST JAVA, INDONESIA: MASS-BALANCE ELUCIDATION USING ARC BASALT SIMULATOR MODEL
ACROSS-ARC GEOCHEMICAL VARIATION OF QUATERNARY LAVAS IN WEST JAVA, INDONESIA: MASS-BALANCE ELUCIDATION USING ARC BASALT SIMULATOR MODEL
ACROSS-ARC VARIATION OF LAVA CHEMISTRY IN THE IZU-BONIN ARC: IDENTIFICATION OF SUBDUCTION COMPONENTS
ACROSS-ARC VARIATIONS IN MO ISOTOPES AND IMPLICATIONS FOR SUBDUCTED OCEANIC CRUST IN THE SOURCE OF BACK-ARC BASIN VOLCANIC ROCKS
"""
memory = {}

如果有查询的一批中有失败的，可以尝试多跑几次，已经成功的不会重复进行查询

In [4]:
out = ""
for line in titles.split('\n'):
    title = line.strip()
    if "" == title:
        continue
    if title in memory:
        apa = memory[title]
    else:
        apa = query_baidu_xueshu(title)
        memory[title] = apa
        time.sleep(1)
    
    out += f'{apa}<br/>'
    
HTML(out)

https://xueshu.baidu.com/s?wd=143ND%2F144ND+AND+87SR%2F86SR+RATIOS+IN+RECENT+MAGMATIC+ROCKS+OF+THE+KURILE+ISLAND+ARC
https://xueshu.baidu.com/u/citation?type=cite&paperid=4f38a87a6cbb0fe8469a8e6a76522da2
https://xueshu.baidu.com/s?wd=A+RHYOLITE+TO+DACITE+SEQUENCE+OF+VOLCANISM+DIRECTLY+FROM+THE+HEATED+LOWER+CRUST%3A+LATE+PLEISTOCENE+TO+HOLOCENE+NUMAZAWA+VOLCANO%2C+NE+JAPAN
https://xueshu.baidu.com/u/citation?type=cite&paperid=ffe55b937ebff878fa26532ff7ba9272
https://xueshu.baidu.com/s?wd=A+TRIASSIC+TO+JURASSIC+ARC+IN+NORTH+BORNEO%3A+GEOCHRONOLOGY%2C+GEOCHEMISTRY%2C+AND+GENESIS+OF+THE+SEGAMA+VALLEY+FELSIC+INTRUSIONS+AND+THE+SABAH+OPHIOLITE
https://xueshu.baidu.com/u/citation?type=cite&paperid=1c6m0py0e8410ty084330c00h3713658
https://xueshu.baidu.com/s?wd=A+U-SERIES+STUDY+OF+LAVAS+FROM+KAMCHATKA+AND+THE+ALEUTIANS%3A+CONSTRAINTS+ON+SOURCE+COMPOSITION+AND+MELTING+PROCESSES
https://xueshu.baidu.com/u/citation?type=cite&paperid=cdb012936da65c9acd813ca4714e4d7f
https://xueshu.baidu.com/s?wd=AC